In [ ]:
# CÉLULA 1: Conectar ao Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# CÉLULA 2 (VERSÃO MULTI-ANO): Carregar e Combinar Dados do Google Drive

import pandas as pd
import os # Biblioteca para lidar com nomes de arquivos e pastas

# Caminho para a pasta onde estão TODOS os arquivos CSV no seu Google Drive
pasta_dados = '/content/drive/MyDrive/mini-projeto/'

print(f"Procurando arquivos CSV na pasta: {pasta_dados}")

# Lista dos anos que queremos carregar (2020 a 2023)
anos_para_carregar = ['20', '21', '22', '23']

# Lista das colunas que realmente vamos usar (Incluindo NU_ANO)
colunas_para_usar = [
    'NU_ANO', # Ano da notificação é essencial agora
    'ID_MUNICIP',
    'SG_UF_NOT',
    'CS_SEXO',
    'DT_NOTIFIC',
    'NU_IDADE_N',
    'CLASSI_FIN',
    'CRITERIO'
]

lista_dfs = [] # Lista vazia para guardar os dados de cada ano

# Loop para carregar cada arquivo
for ano in anos_para_carregar:
    nome_arquivo = f'DENGBR{ano}.csv'
    caminho_completo = os.path.join(pasta_dados, nome_arquivo) # Monta o caminho completo: /content/.../DENGBR20.csv, etc.
    print(f"Tentando carregar: {caminho_completo}...")

    try:
        # Lê o CSV do ano atual, apenas as colunas necessárias
        df_ano = pd.read_csv(
            caminho_completo,
            sep=',',
            encoding='latin-1',
            usecols=colunas_para_usar,
            on_bad_lines='skip',
            low_memory=False
        )
        # Adiciona o DataFrame lido à nossa lista
        lista_dfs.append(df_ano)
        print(f" -> Carregado com sucesso ({len(df_ano)} linhas)")

    except FileNotFoundError:
        print(f" -> ❌ ERRO: Arquivo '{nome_arquivo}' não encontrado! Pulando este ano.")
    except ValueError as ve:
        # Este erro acontece se alguma coluna de 'colunas_para_usar' não existir no arquivo
        print(f" -> ❌ ERRO ao ler colunas em '{nome_arquivo}': {ve}. Verifique se as colunas estão corretas.")
    except Exception as e:
        print(f" -> ❌ Ocorreu um erro inesperado ao carregar '{nome_arquivo}': {e}")

# Verifica se conseguimos carregar algum arquivo antes de tentar combinar
if lista_dfs:
    # Combina todos os DataFrames da lista em um único DataFrame grande
    df_dengue_total = pd.concat(lista_dfs, ignore_index=True)
    print("\n✅ Todos os arquivos disponíveis foram combinados com sucesso!")
    print(f"Total de linhas carregadas: {len(df_dengue_total)}")
    display(df_dengue_total.head()) # Mostra as primeiras linhas do DataFrame combinado
    print("\n--- Informações técnicas do DataFrame combinado: ---")
    df_dengue_total.info() # Mostra informações sobre o DataFrame combinado
else:
    print("\n❌ Nenhum arquivo de dados foi carregado. Verifique os caminhos e nomes dos arquivos na pasta do Drive.")
    # Cria um DataFrame vazio para evitar erros nas próximas células
    df_dengue_total = pd.DataFrame(columns=colunas_para_usar)

Procurando arquivos CSV na pasta: /content/drive/MyDrive/mini-projeto/
Tentando carregar: /content/drive/MyDrive/mini-projeto/DENGBR20.csv...
 -> Carregado com sucesso (1495117 linhas)
Tentando carregar: /content/drive/MyDrive/mini-projeto/DENGBR21.csv...
 -> Carregado com sucesso (1010359 linhas)
Tentando carregar: /content/drive/MyDrive/mini-projeto/DENGBR22.csv...
 -> Carregado com sucesso (1393877 linhas)
Tentando carregar: /content/drive/MyDrive/mini-projeto/DENGBR23.csv...
 -> Carregado com sucesso (1508653 linhas)

✅ Todos os arquivos disponíveis foram combinados com sucesso!
Total de linhas carregadas: 5408006


,DT_NOTIFIC,NU_ANO,SG_UF_NOT,ID_MUNICIP,NU_IDADE_N,CS_SEXO,CLASSI_FIN,CRITERIO
0,2020-01-26,2020,12,120020,4023.0,F,5.0,2.0
1,2020-01-30,2020,12,120020,4038.0,F,5.0,2.0
2,2020-02-11,2020,12,120020,4020.0,F,5.0,1.0
3,2019-12-30,2019,12,120020,4037.0,F,5.0,2.0
4,2020-02-07,2020,13,130165,4020.0,F,5.0,1.0



--- Informações técnicas do DataFrame combinado: ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5408006 entries, 0 to 5408005
Data columns (total 8 columns):
 #   Column      Dtype  
---  ------      -----  
 0   DT_NOTIFIC  object 
 1   NU_ANO      int64  
 2   SG_UF_NOT   int64  
 3   ID_MUNICIP  int64  
 4   NU_IDADE_N  float64
 5   CS_SEXO     object 
 6   CLASSI_FIN  float64
 7   CRITERIO    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 330.1+ MB


In [ ]:
# CÉLULA DE LIMPEZA (CORRIGIDA - USA df_dengue_total)

print("Iniciando a limpeza e preparação dos dados...")

try:
    # Cria uma cópia do DataFrame COMBINADO para trabalhar
    df_limpo = df_dengue_total.copy() # <-- CORREÇÃO APLICADA AQUI!

    # Processa a coluna de idade 'NU_IDADE_N' para extrair a idade em anos
    df_limpo['NU_IDADE_N'] = pd.to_numeric(df_limpo['NU_IDADE_N'], errors='coerce')
    df_limpo.dropna(subset=['NU_IDADE_N'], inplace=True)
    df_limpo = df_limpo[df_limpo['NU_IDADE_N'] >= 4000]
    df_limpo['IDADE'] = df_limpo['NU_IDADE_N'] - 4000
    print("- Coluna 'IDADE' processada com sucesso.")

    # Converte a coluna NU_ANO para inteiro (importante para filtros)
    if 'NU_ANO' in df_limpo.columns:
         df_limpo['NU_ANO'] = df_limpo['NU_ANO'].astype(int)
         print("- Coluna 'NU_ANO' verificada/convertida.")

    print("\n✅ Processo de limpeza concluído!")
    display(df_limpo.head())

except NameError:
    print("\n❌ ERRO: A variável 'df_dengue_total' não foi criada. Verifique se a Célula 2 (carregamento multi-ano) foi executada sem erros.")
except Exception as e:
    print(f"\n❌ Ocorreu um erro inesperado durante a limpeza: {e}")

Iniciando a limpeza e preparação dos dados...
- Coluna 'IDADE' processada com sucesso.
- Coluna 'NU_ANO' verificada/convertida.

✅ Processo de limpeza concluído!


,DT_NOTIFIC,NU_ANO,SG_UF_NOT,ID_MUNICIP,NU_IDADE_N,CS_SEXO,CLASSI_FIN,CRITERIO,IDADE
0,2020-01-26,2020,12,120020,4023.0,F,5.0,2.0,23.0
1,2020-01-30,2020,12,120020,4038.0,F,5.0,2.0,38.0
2,2020-02-11,2020,12,120020,4020.0,F,5.0,1.0,20.0
3,2019-12-30,2019,12,120020,4037.0,F,5.0,2.0,37.0
4,2020-02-07,2020,13,130165,4020.0,F,5.0,1.0,20.0


In [ ]:
# CÉLULA 3: Análise Exploratória dos Dados

try:
    print("Iniciando a análise dos dados limpos...")

    # Pergunta 1: Quantos casos de dengue foram notificados por estado?
    print("\n--- Contagem de Casos por Estado ---")
    casos_por_estado = df_limpo['SG_UF_NOT'].value_counts()
    print(casos_por_estado)

    # Pergunta 2: Qual a distribuição de casos por sexo?
    print("\n--- Contagem de Casos por Sexo ---")
    casos_por_sexo = df_limpo['CS_SEXO'].value_counts()
    print(casos_por_sexo)

    # Pergunta 3: Quais foram os critérios de confirmação mais comuns?
    print("\n--- Contagem por Critério de Confirmação ---")
    casos_por_criterio = df_limpo['CRITERIO'].value_counts()
    print(casos_por_criterio)

    # Pergunta 4: Qual a média de idade dos pacientes notificados?
    print(f"\n--- Idade Média dos Pacientes ---")
    idade_media = df_limpo['IDADE'].mean()
    print(f"A idade média dos pacientes com dengue notificados em 2023 foi de {idade_media:.1f} anos.")

except NameError:
    print("\n❌ ERRO: A variável 'df_limpo' não foi criada. Verifique se a célula de limpeza (anterior a esta) foi executada sem erros.")
except Exception as e:
    print(f"\n❌ Ocorreu um erro inesperado durante a análise: {e}")

Iniciando a análise dos dados limpos...

--- Contagem de Casos por Estado ---
SG_UF_NOT
35    1421487
41     804206
31     687328
52     438272
42     284096
29     222940
53     202306
50     167869
23     164751
51     142251
43     128077
26     118923
33      74133
24      68701
25      65321
12      55508
27      51100
22      46387
17      46229
13      36262
11      35069
15      27205
28      17656
21      16096
32       7779
14       3744
16       2703
Name: count, dtype: int64

--- Contagem de Casos por Sexo ---
CS_SEXO
F    2897551
M    2432787
I       6046
Name: count, dtype: int64

--- Contagem por Critério de Confirmação ---
CRITERIO
2.0    2424407
1.0    2295891
3.0      46247
Name: count, dtype: int64

--- Idade Média dos Pacientes ---
A idade média dos pacientes com dengue notificados em 2023 foi de 34.9 anos.


In [ ]:
# CÉLULA 4 (COM DEPURAÇÃO): Visualização dos Dados

import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact
import traceback # Para mostrar erros mais detalhados

print("--- Configurando Visualização Interativa por Ano ---")

if 'NU_ANO' in df_limpo.columns:
    anos_disponiveis = sorted(df_limpo['NU_ANO'].unique())
    opcoes_dropdown = [('Todos os Anos', 0)] + [(str(ano), ano) for ano in anos_disponiveis]

    year_selector = widgets.Dropdown(
        options=opcoes_dropdown, value=0, description='Selecione o Ano:', disabled=False,
    )

    # Função que gera o gráfico
    def plot_casos_por_estado_por_ano(ano_selecionado):
        try: # Adicionamos um try/except aqui dentro
            print(f"\n-> Tentando gerar gráfico para o ano: {ano_selecionado if ano_selecionado != 0 else 'Todos'}") # DEBUG
            if ano_selecionado == 0:
                df_filtrado_ano = df_limpo
                titulo = f'Total de Casos (2020-2023)' # Título mais curto
            else:
                df_filtrado_ano = df_limpo[df_limpo['NU_ANO'] == ano_selecionado]
                titulo = f'Casos em {ano_selecionado}'

            if df_filtrado_ano.empty:
                print(" -> Nenhum dado encontrado para este ano.")
                return

            print(" -> Calculando contagem por estado...") # DEBUG
            casos_por_estado = df_filtrado_ano['SG_UF_NOT'].value_counts().reset_index()
            casos_por_estado.columns = ['Estado_Codigo', 'Numero_de_Casos']

            print(" -> Mapeando siglas dos estados...") # DEBUG
            mapa_uf = { 11: 'RO', 12: 'AC', 13: 'AM', 14: 'RR', 15: 'PA', 16: 'AP', 17: 'TO', 21: 'MA', 22: 'PI', 23: 'CE', 24: 'RN', 25: 'PB', 26: 'PE', 27: 'AL', 28: 'SE', 29: 'BA', 31: 'MG', 32: 'ES', 33: 'RJ', 35: 'SP', 41: 'PR', 42: 'SC', 43: 'RS', 50: 'MS', 51: 'MT', 52: 'GO', 53: 'DF' }
            casos_por_estado['Sigla_Estado'] = casos_por_estado['Estado_Codigo'].map(mapa_uf)
            # Verifica se algum estado não foi mapeado (ficou NaN)
            if casos_por_estado['Sigla_Estado'].isnull().any():
                print(" -> ATENÇÃO: Alguns códigos de estado não foram mapeados para siglas!")

            print(" -> Criando a figura do gráfico...") # DEBUG
            fig = px.bar(
                casos_por_estado.sort_values('Numero_de_Casos', ascending=False),
                x='Sigla_Estado', y='Numero_de_Casos', title=titulo,
                labels={'Sigla_Estado': 'Estado', 'Numero_de_Casos': 'Notificações'},
                template='plotly_white'
            )
            print(" -> Mostrando o gráfico.") # DEBUG
            fig.show()
            print(" -> Gráfico exibido com sucesso!") # DEBUG

        except Exception as e:
            print(f"\n❌ ERRO DENTRO DA FUNÇÃO DE PLOTAGEM:")
            traceback.print_exc() # Mostra o erro detalhado

    # Conecta o widget à função
    interact(plot_casos_por_estado_por_ano, ano_selecionado=year_selector)

else:
    print("❌ ERRO: A coluna 'NU_ANO' não foi encontrada no DataFrame 'df_limpo'.")

--- Configurando Visualização Interativa por Ano ---


interactive(children=(Dropdown(description='Selecione o Ano:', options=(('Todos os Anos', 0), ('2019', np.int6…

In [ ]:
# CÉLULA ANTES DO WRITEFILE: Salva os dados limpos localmente

print("Salvando DataFrame limpo para uso do Streamlit...")
try:
    caminho_local_csv = '/content/dados_limpos_para_app.csv'
    df_limpo.to_csv(caminho_local_csv, index=False, encoding='latin-1', sep=',')
    print(f"✅ Dados limpos salvos com sucesso em: {caminho_local_csv}")
except NameError:
    print("❌ ERRO: A variável 'df_limpo' não existe. Execute a célula de limpeza primeiro.")
except Exception as e:
    print(f"❌ ERRO ao salvar o arquivo local: {e}")

Salvando DataFrame limpo para uso do Streamlit...
✅ Dados limpos salvos com sucesso em: /content/dados_limpos_para_app.csv


In [ ]:
# CÉLULA %%writefile app.py (VERSÃO MULTI-ANO COM FILTRO DE ANO)

%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

# --- CONFIGURAÇÃO DA PÁGINA ---
st.set_page_config(page_title="Análise de Dengue no Brasil (Multi-Ano)", page_icon="🦟", layout="wide")

# --- ESTILO CSS ---
st.markdown("""<style> [data-testid="stSidebar"] { background-color: #ADD8E6; } footer {visibility: hidden;} </style>""", unsafe_allow_html=True)

# --- FUNÇÃO DE CARREGAMENTO (Lê o CSV multi-ano salvo localmente) ---
@st.cache_data
def carregar_dados_locais():
    try:
        caminho_local_csv = '/content/dados_limpos_para_app.csv'
        df = pd.read_csv(caminho_local_csv, sep=',', encoding='latin-1')
        # Prepara colunas de mapeamento
        mapa_uf = { 11:'RO', 12:'AC', 13:'AM', 14:'RR', 15:'PA', 16:'AP', 17:'TO', 21:'MA', 22:'PI', 23:'CE', 24:'RN', 25:'PB', 26:'PE', 27:'AL', 28:'SE', 29:'BA', 31:'MG', 32:'ES', 33:'RJ', 35:'SP', 41:'PR', 42:'SC', 43:'RS', 50:'MS', 51:'MT', 52:'GO', 53:'DF' }
        df['ESTADO'] = df['SG_UF_NOT'].map(mapa_uf)
        mapa_sexo = {'M': 'Masculino', 'F': 'Feminino', 'I': 'Ignorado'}
        df['SEXO'] = df['CS_SEXO'].map(mapa_sexo)
        mapa_criterio = {1.0: 'Laboratorial', 2.0: 'Clínico-Epidem.', 3.0: 'Em Investigação'}
        df['CRITERIO_DESC'] = df['CRITERIO'].map(mapa_criterio)
        # Garante que colunas importantes não tenham NaNs e que NU_ANO seja inteiro
        df.dropna(subset=['ESTADO', 'NU_ANO'], inplace=True)
        df['NU_ANO'] = df['NU_ANO'].astype(int)
        return df
    except FileNotFoundError:
        st.error("ERRO: O arquivo 'dados_limpos_para_app.csv' não foi encontrado.")
        return None
    except Exception as e:
        st.error(f"Erro ao carregar/processar dados locais: {e}")
        return None

# --- CARREGA OS DADOS ---
df_analise = carregar_dados_locais()

# --- BARRA LATERAL (SIDEBAR) ---
st.sidebar.header("Filtros")
try:
    st.sidebar.image("https://logodownload.org/wp-content/uploads/2014/02/sus-logo-0.png", width=100)
    st.sidebar.image("https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Flag_of_Brazil.svg/200px-Flag_of_Brazil.svg.png", width=100)
except Exception:
    pass

if df_analise is not None:
    anos_disponiveis = sorted(df_analise['NU_ANO'].unique())
    # === ADIÇÃO DO FILTRO DE ANO ===
    anos_selecionados = st.sidebar.multiselect(
        "Selecione o(s) Ano(s):",
        options=anos_disponiveis,
        default=anos_disponiveis # Começa com todos os anos selecionados
    )
    # ================================

    lista_estados = sorted(df_analise['ESTADO'].unique())
    estados_selecionados = st.sidebar.multiselect(
        "Selecione o(s) Estado(s) (Opcional):",
        options=lista_estados,
        default=[]
    )

    # Aplica os filtros de ANO e ESTADO
    if not anos_selecionados: # Garante que pelo menos um ano esteja selecionado
        anos_selecionados = anos_disponiveis
    # Filtra primeiro por ANO
    df_filtrado = df_analise[df_analise['NU_ANO'].isin(anos_selecionados)]

    # Depois filtra por ESTADO (se algum foi selecionado)
    if estados_selecionados:
        df_filtrado = df_filtrado[df_filtrado['ESTADO'].isin(estados_selecionados)]
        titulo_local = f"{', '.join(estados_selecionados)}"
    else:
        titulo_local = "Brasil"

    titulo_anos = f"({min(anos_selecionados)}-{max(anos_selecionados)})" if len(anos_selecionados) > 1 else f"({anos_selecionados[0]})"
    titulo_principal = f"Análise de Dengue - {titulo_local} {titulo_anos}"

else:
    df_filtrado = pd.DataFrame()
    titulo_principal = "Análise de Dengue - Erro ao carregar dados"
    st.sidebar.error("Dados não carregados, filtros desativados.")

# --- INTERFACE PRINCIPAL DA APLICAÇÃO ---
st.title(f"🦟 {titulo_principal}")
st.markdown("Análise interativa dos dados de notificação de Dengue do SINAN.")

if not df_filtrado.empty:
    # Gráfico 1: Evolução Temporal
    st.header("Evolução Anual dos Casos (Seleção Atual)")
    casos_por_ano_filtrado = df_filtrado['NU_ANO'].value_counts().sort_index().reset_index()
    casos_por_ano_filtrado.columns = ['Ano', 'Numero_de_Casos']
    fig_linha = px.line(casos_por_ano_filtrado, x='Ano', y='Numero_de_Casos', title='Casos Notificados por Ano', markers=True)
    st.plotly_chart(fig_linha, use_container_width=True)

    # Gráfico 2: Distribuição por Estado
    st.header("Distribuição por Estado (Seleção Atual)")
    casos_estado_filtrado = df_filtrado['ESTADO'].value_counts().reset_index()
    casos_estado_filtrado.columns = ['Estado', 'Numero_de_Casos']
    fig_barra_estado = px.bar(casos_estado_filtrado.sort_values('Numero_de_Casos', ascending=False), x='Estado', y='Numero_de_Casos', title='Casos por Estado', labels={'Estado': 'Estado', 'Numero_de_Casos': 'Total de Notificações'})
    st.plotly_chart(fig_barra_estado, use_container_width=True)

    # Gráficos Adicionais
    st.header("Análises Adicionais (Seleção Atual)")
    col1, col2 = st.columns(2)
    with col1:
        st.subheader("Distribuição por Sexo")
        casos_sexo = df_filtrado['SEXO'].value_counts()
        st.bar_chart(casos_sexo)
    with col2:
        st.subheader("Critério de Confirmação")
        casos_criterio = df_filtrado['CRITERIO_DESC'].value_counts()
        st.bar_chart(casos_criterio)

    if st.checkbox("Mostrar tabela de dados filtrados"):
        st.dataframe(df_filtrado[['NU_ANO', 'ESTADO', 'SEXO', 'IDADE', 'CRITERIO_DESC']]) # Mostra colunas mais legíveis

else:
     if df_analise is not None:
       st.warning("Nenhum dado encontrado para os filtros selecionados.")

Overwriting app.py


In [ ]:
# CÉLULA 6: Instala as bibliotecas e executa a aplicação

!pip install -q streamlit pyngrok

from pyngrok import ngrok
import time
import subprocess

# Coloque aqui o seu Authtoken que você já tem do site ngrok.com
AUTHTOKEN = "346zypbvE9ocMsQSJja8KsrtB4m_Q2ojQaYe6TtqBNqfRyHX"

# Configura o token e executa o dashboard
ngrok.set_auth_token(AUTHTOKEN)
ngrok.kill()
!pkill streamlit
time.sleep(5)

# Inicia o streamlit em segundo plano
process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port=8501'])
time.sleep(10) # Damos um tempo para o servidor iniciar completamente

# Cria o link público
try:
    public_url = ngrok.connect(8501)
    print("------------------------------------------------------------------")
    print(f"🎉 SEU DASHBOARD ESTÁ PRONTO! Acesse pelo link: {public_url}")
    print("------------------------------------------------------------------")
except Exception as e:
    print(f"❌ Falha ao criar o túnel do ngrok: {e}")

------------------------------------------------------------------
🎉 SEU DASHBOARD ESTÁ PRONTO! Acesse pelo link: NgrokTunnel: "https://unpuritanical-elliptical-katalina.ngrok-free.dev" -> "http://localhost:8501"
------------------------------------------------------------------
